In [ ]:
# 1. SETUP

import os, gc, torch
import torch.nn.functional as F
import numpy as np
import pandas as pd
from glob import glob
from PIL import Image
from tqdm import tqdm
from torchvision import transforms, datasets
import timm

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print("DEVICE:", DEVICE)

TRAIN_DIR = "/kaggle/input/action-dm-dataset/action-dm-dataset_2/train"
TEST_DIR  = "/kaggle/input/action-dm-dataset/action-dm-dataset_2/test/test"
OUT_FILE  = "submission_ensemble_rank4_boosted.csv"

MODEL_PATHS = [
    "/kaggle/input/saved-best-models/swin_large_mixcut_ema_best.pth",
    "/kaggle/input/saved-best-models/best_model_fold_0.pth",
    "/kaggle/input/saved-best-models/best_model_fold_1.pth",
    "/kaggle/input/saved-best-models/best_model_fold_2.pth",
]

BACKBONES = [
    "swinv2_large_window12to24_192to384_22kft1k",
    "convnextv2_large.fcmae_ft_in22k_in1k",
    "convnextv2_large.fcmae_ft_in22k_in1k",
    "convnextv2_large.fcmae_ft_in22k_in1k",
]

VAL_ACCS = [0.9190, 0.9118, 0.9201, 0.9239]
weights = torch.tensor(VAL_ACCS) / sum(VAL_ACCS)
weights = weights.to(DEVICE)

IMG_SIZE = 384
BATCH_SIZE = 8
TEMPERATURE = 1.6

gc.collect()
torch.cuda.empty_cache()

In [ ]:
# 2. IMPROVED TTA (lebih natural)

tta_tfms = [
    transforms.Compose([
        transforms.Resize(int(IMG_SIZE * 1.14)),
        transforms.CenterCrop(IMG_SIZE),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ]),
    transforms.Compose([
        transforms.Resize(int(IMG_SIZE * 1.14)),
        transforms.CenterCrop(IMG_SIZE),
        transforms.RandomHorizontalFlip(p=1.0),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ]),
    transforms.Compose([
        transforms.Resize(int(IMG_SIZE * 1.14)),
        transforms.CenterCrop(IMG_SIZE),
        transforms.ColorJitter(brightness=0.1, contrast=0.1),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ]),
    transforms.Compose([
        transforms.RandomResizedCrop(IMG_SIZE, scale=(0.92, 1.0)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ])
]

In [ ]:
# 3. LOAD CLASSES & MODELS

trainset = datasets.ImageFolder(TRAIN_DIR)
CLASSES = trainset.classes
n_classes = len(CLASSES)

def load_model(backbone, num_classes, path):
    print(f"Loading {backbone} from {os.path.basename(path)}")
    model = timm.create_model(backbone, pretrained=False, num_classes=num_classes)
    state = torch.load(path, map_location=DEVICE)
    model.load_state_dict(state, strict=False)
    model.to(DEVICE)
    model.eval()
    return model

models = [load_model(bb, n_classes, p) for bb, p in zip(BACKBONES, MODEL_PATHS)]

torch.backends.cudnn.benchmark = True
torch.set_grad_enabled(False)

In [ ]:
# 4. INFERENCE + GEOMETRIC ENSEMBLE

test_imgs = sorted(glob(os.path.join(TEST_DIR, "*")))
results = []

with torch.no_grad(), torch.cuda.amp.autocast():
    for img_path in tqdm(test_imgs, desc="Inferencing (Boosted Rank 4 Ensemble)"):
        img = Image.open(img_path).convert("RGB")

        probs_per_model = []
        for model in models:
            probs_tta = []
            for tta in tta_tfms:
                x = tta(img).unsqueeze(0).to(DEVICE, non_blocking=True)
                logits = model(x)
                probs = F.softmax(logits / TEMPERATURE, dim=1)
                probs_tta.append(probs)
            probs_tta = torch.stack(probs_tta).mean(0)
            probs_per_model.append(probs_tta)

        probs_per_model = torch.stack(probs_per_model)
        weighted_log_probs = torch.log(probs_per_model + 1e-8) * weights[:, None, None]
        geo_mean = torch.exp(weighted_log_probs.sum(0))
        probs = geo_mean / geo_mean.sum(dim=1, keepdim=True)

        idx = torch.argmax(probs, dim=1).item()
        label = CLASSES[idx]
        results.append({"ID": os.path.basename(img_path), "label": label})

In [ ]:
# 5. SAVE SUBMISSION

df = pd.DataFrame(results)
df.to_csv(OUT_FILE, index=False)
print(f"✅ Saved boosted ensemble submission to {OUT_FILE}")